# MobAI'26 - Training Notebook
## Task 1: Warehouse Optimization & Task 2: Demand Forecasting

This notebook documents the complete training pipeline for both tasks:
- **Task 2 (Forecasting)**: Prophet per-SKU + XGBoost Classifier + Blend
- **Task 1 (Optimization)**: Heuristic-based warehouse optimization (no ML model needed)

### Architecture Summary
```
Stage 0: Prophet per-SKU (cps=0.01, sps=0.1, multiplicative, 5 temporal regressors)
Stage 1: XGBoost Classifier (demand yes/no, AUC=0.9375)
Stage 2: Threshold + Blend + Bias tuning (WAPE=73.88%, Bias=0.62%)
```

### Key Results
| Metric | Value |
|--------|-------|
| WAPE (demand-days) | 73.88% |
| Bias (demand-days) | 0.62% |
| Classifier AUC | 0.9375 |
| Improvement vs Lag-1 | 31.7% |
| Improvement vs EWM-7 | 11.1% |

## 1. Setup & Data Loading

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from pathlib import Path
from prophet import Prophet
from sklearn.metrics import roc_auc_score
import json, warnings, time

warnings.filterwarnings("ignore")
import logging
logging.getLogger("prophet").setLevel(logging.WARNING)
logging.getLogger("cmdstanpy").setLevel(logging.WARNING)
np.random.seed(42)

BASE  = Path(".")  # Adjust if needed
DATA  = BASE / "data"
MODEL = BASE / "models"
XLSX  = DATA / "WMS_Hackathon_DataPack_Templates_FR_FV_B7_ONLY.xlsx"

print(f"Data directory: {DATA.resolve()}")
print(f"Model directory: {MODEL.resolve()}")

In [ ]:
# Load all data sources
demand_raw = pd.read_excel(XLSX, "historique_demande")
products   = pd.read_csv(DATA / "product_priorities.csv")
segments   = pd.read_csv(DATA / "product_segments.csv")

# Raw products table (physical attributes)
products_raw = pd.read_excel(XLSX, "produits", header=0, skiprows=[1, 2])
products_raw = products_raw[["id_produit", "categorie", "colisage fardeau",
                              "colisage palette", "volume pcs (m3)",
                              "Poids(kg)", "Is_Gerbable"]].copy()
products_raw.columns = ["id_produit", "categorie_raw", "colisage_fardeau",
                         "colisage_palette", "volume_pcs", "poids_kg",
                         "is_gerbable"]
products_raw["is_gerbable"] = products_raw["is_gerbable"].map(
    {True: 1.0, "True": 1.0, False: 0.0, "False": 0.0}
).fillna(0.0)
products_raw["volume_pcs"]       = pd.to_numeric(products_raw["volume_pcs"], errors="coerce").fillna(0)
products_raw["colisage_fardeau"] = pd.to_numeric(products_raw["colisage_fardeau"], errors="coerce").fillna(1)
products_raw["colisage_palette"] = pd.to_numeric(products_raw["colisage_palette"], errors="coerce").fillna(1)
products_raw["poids_kg"]         = pd.to_numeric(products_raw["poids_kg"], errors="coerce").fillna(0)

# Transactions (delivery patterns)
transactions = pd.read_excel(XLSX, "transactions", header=0, skiprows=[1, 2])
trans_lines  = pd.read_excel(XLSX, "lignes_transaction", header=0, skiprows=[1, 2])

print(f"Demand records: {len(demand_raw):,}")
print(f"Products: {len(products)}")
print(f"Segments: {segments['segment'].value_counts().to_dict()}")
print(f"Transactions: {len(transactions):,}")

## 2. Data Exploration

In [ ]:
# Explore demand data
demand_raw["date"] = pd.to_datetime(demand_raw["date"])
demand_raw["day"]  = demand_raw["date"].dt.normalize()

print("=== Demand Data ===")
print(f"Date range: {demand_raw['day'].min().date()} to {demand_raw['day'].max().date()}")
print(f"Unique products: {demand_raw['id_produit'].nunique()}")
print(f"Total records: {len(demand_raw):,}")
print(f"\nDemand statistics:")
print(demand_raw['quantite_demande'].describe())

# Daily aggregation
daily = (
    demand_raw.groupby(["day", "id_produit"])["quantite_demande"]
    .sum().reset_index()
    .rename(columns={"quantite_demande": "demand"})
)
daily["demand"] = daily["demand"].clip(lower=0)

# Outlier capping (99th percentile per SKU)
cap = daily.groupby("id_produit")["demand"].quantile(0.99).rename("cap99")
daily = daily.merge(cap, on="id_produit", how="left")
daily["demand"] = daily[["demand", "cap99"]].min(axis=1)
daily.drop(columns="cap99", inplace=True)

all_prods = daily["id_produit"].unique()
all_dates = pd.date_range(daily["day"].min(), daily["day"].max(), freq="D")
print(f"\nAfter aggregation: {len(daily):,} product-day records")
print(f"Products: {len(all_prods)}, Days: {len(all_dates)}")

In [ ]:
# Sparsity analysis
total_cells = len(all_prods) * len(all_dates)
demand_cells = len(daily)
sparsity = 1 - demand_cells / total_cells

print(f"Full grid size: {total_cells:,} cells")
print(f"Cells with demand: {demand_cells:,}")
print(f"Sparsity: {sparsity:.1%}")

# Demand frequency distribution
prod_freq = daily.groupby("id_produit")["day"].nunique() / len(all_dates)
print(f"\nDemand frequency distribution:")
print(prod_freq.describe())
print(f"\nHigh frequency (>5% days): {(prod_freq > 0.05).sum()} products")
print(f"Low frequency (<1% days): {(prod_freq < 0.01).sum()} products")

## 3. Transaction-Derived Features

In [ ]:
# Build delivery pattern features from transactions
transactions["cree_le"] = pd.to_datetime(transactions["cree_le"])
deliveries = transactions[transactions["type_transaction"] == "DELIVERY"].copy()
deliveries["day"] = deliveries["cree_le"].dt.normalize()

del_lines = trans_lines.merge(
    deliveries[["id_transaction", "day"]], on="id_transaction", how="inner"
)

daily_del = (
    del_lines.groupby(["day", "id_produit"])
    .agg(n_deliveries=("quantite", "count"),
         qty_delivered=("quantite", "sum"))
    .reset_index()
)
daily_del["qty_delivered"] = daily_del["qty_delivered"].clip(lower=0)

prod_del_stats = (
    daily_del.groupby("id_produit")
    .agg(
        del_total_count=("n_deliveries", "sum"),
        del_total_qty=("qty_delivered", "sum"),
        del_n_days=("day", "nunique"),
        del_avg_qty=("qty_delivered", "mean"),
    )
    .reset_index()
)

print(f"Delivery records: {len(daily_del):,}")
print(f"Products with deliveries: {del_lines['id_produit'].nunique()}")

## 4. Full Calendar Grid Construction

In [ ]:
# Build full grid (every product x every day)
grid = pd.MultiIndex.from_product(
    [all_dates, all_prods], names=["day", "id_produit"]
).to_frame(index=False)
grid = grid.merge(daily, on=["day", "id_produit"], how="left")
grid["demand"] = grid["demand"].fillna(0)
grid["has_demand"] = (grid["demand"] > 0).astype(int)

# Merge delivery counts
grid = grid.merge(daily_del[["day", "id_produit", "n_deliveries", "qty_delivered"]],
                  on=["day", "id_produit"], how="left")
grid["n_deliveries"]  = grid["n_deliveries"].fillna(0)
grid["qty_delivered"]  = grid["qty_delivered"].fillna(0)

# Merge segments and physical attributes
grid = grid.merge(segments, on="id_produit", how="left")
grid["segment"] = grid["segment"].fillna("LF")
grid["is_hf"]   = (grid["segment"] == "HF").astype(float)

phys_cols = ["id_produit", "colisage_fardeau", "colisage_palette",
             "volume_pcs", "poids_kg", "is_gerbable"]
grid = grid.merge(products_raw[phys_cols], on="id_produit", how="left")
for c in ["colisage_fardeau", "colisage_palette", "volume_pcs", "poids_kg", "is_gerbable"]:
    grid[c] = grid[c].fillna(0)

grid = grid.merge(prod_del_stats, on="id_produit", how="left")
for c in ["del_total_count", "del_total_qty", "del_n_days", "del_avg_qty"]:
    grid[c] = grid[c].fillna(0)

grid.sort_values(["id_produit", "day"], inplace=True)
grid.reset_index(drop=True, inplace=True)

print(f"Grid shape: {grid.shape}")
print(f"Sparsity: {(grid['demand']==0).mean():.1%}")
print(f"HF SKUs: {grid.loc[grid['is_hf']==1, 'id_produit'].nunique()}")
print(f"LF SKUs: {grid.loc[grid['is_hf']==0, 'id_produit'].nunique()}")

## 5. Stage 0: Prophet Per-SKU with Temporal Regressors

### Model Configuration (Proven from Kaggle)
- `changepoint_prior_scale = 0.01` (smooth trend)
- `seasonality_prior_scale = 0.1` (moderate seasonality)
- `seasonality_mode = "multiplicative"`
- 5 temporal regressors: `day_of_week`, `is_weekend`, `month`, `is_month_start`, `is_month_end`
- Per-product calibration factor (actual_mean / predicted_mean on demand-days)

In [ ]:
# Prophet per-SKU fitting
seg_map = segments.set_index("id_produit")["segment"].to_dict()
grid_prophet = grid[["day", "id_produit", "demand"]].copy()

prod_demand = daily.groupby("id_produit")["demand"].agg(["sum", "count"]).reset_index()
prod_demand.columns = ["id_produit", "total_demand", "n_demand_days"]

MIN_DAYS_PROPHET = 10
prophet_prods = set(prod_demand.loc[
    prod_demand["n_demand_days"] >= MIN_DAYS_PROPHET, "id_produit"
])
simple_prods = set(all_prods) - prophet_prods

prod_freq = (
    grid_prophet.groupby("id_produit")
    .apply(lambda g: (g["demand"] > 0).mean())
    .to_dict()
)

print(f"Products for Prophet fitting: {len(prophet_prods)}")
print(f"Products for simple baseline: {len(simple_prods)}")

In [ ]:
# Fit Prophet models (this takes ~10-15 minutes for all 571 products)
prophet_predictions = {}
prophet_models_meta = {}
t0 = time.time()
fitted_count = 0

for i, prod_id in enumerate(list(prophet_prods)):
    prod_data = grid_prophet[grid_prophet["id_produit"] == prod_id][["day", "demand"]].copy()
    prod_data.columns = ["ds", "y"]
    prod_data = prod_data.sort_values("ds").reset_index(drop=True)
    
    seg = seg_map.get(prod_id, "LF")
    freq = prod_freq.get(prod_id, 0.1)
    
    try:
        m = Prophet(
            growth="linear",
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=False,
            seasonality_mode="multiplicative",
            changepoint_prior_scale=0.01,
            seasonality_prior_scale=0.1,
            interval_width=0.95,
        )
        
        for reg_name in ["day_of_week", "is_weekend", "month",
                         "is_month_start", "is_month_end"]:
            m.add_regressor(reg_name, standardize=True)
        
        prod_data["day_of_week"] = prod_data["ds"].dt.dayofweek.astype(float)
        prod_data["is_weekend"]  = (prod_data["day_of_week"] >= 5).astype(float)
        prod_data["month"]       = prod_data["ds"].dt.month.astype(float)
        prod_data["is_month_start"] = prod_data["ds"].dt.is_month_start.astype(float)
        prod_data["is_month_end"]   = prod_data["ds"].dt.is_month_end.astype(float)
        
        m.fit(prod_data)
        forecast = m.predict(prod_data[["ds", "day_of_week", "is_weekend",
                                        "month", "is_month_start", "is_month_end"]])
        
        yhat_raw = forecast["yhat"].clip(lower=0).values
        trend = forecast["trend"].values
        weekly = forecast.get("weekly", pd.Series(0, index=forecast.index)).values
        yearly = forecast.get("yearly", pd.Series(0, index=forecast.index)).values
        
        # Per-product calibration
        demand_mask = prod_data["y"].values > 0
        if demand_mask.sum() > 0:
            actual_dd_mean = prod_data.loc[demand_mask, "y"].mean()
            pred_dd_mean = yhat_raw[demand_mask].mean()
            cal = actual_dd_mean / pred_dd_mean if pred_dd_mean > 0.01 else 1.0
            cal = np.clip(cal, 0.2, 5.0)
        else:
            cal = 1.0
        
        yhat = yhat_raw * cal
        
        for j, (d, _) in enumerate(zip(prod_data["ds"].values, yhat)):
            prophet_predictions[(d, prod_id)] = {
                "yhat": float(yhat[j]),
                "trend": float(trend[j]),
                "weekly": float(weekly[j]),
                "yearly": float(yearly[j]),
            }
        
        prophet_models_meta[int(prod_id)] = {
            "segment": seg,
            "mean_yhat": float(np.mean(yhat)),
            "trend_slope": float((trend[-1] - trend[0]) / max(len(trend), 1)),
            "cal_factor": float(cal),
            "demand_freq": float(freq),
        }
        fitted_count += 1
    except Exception:
        simple_prods.add(prod_id)
        prophet_prods.discard(prod_id)
    
    if (i + 1) % 100 == 0:
        elapsed = time.time() - t0
        print(f"   {i+1}/{len(prophet_prods)} fitted ({elapsed:.0f}s)")

print(f"\nFitted {fitted_count} Prophet models in {time.time()-t0:.0f}s")

# Simple baseline for remaining products
simple_avg = {}
for prod_id in simple_prods:
    pdf = grid_prophet.loc[grid_prophet["id_produit"] == prod_id, "demand"]
    dd = pdf[pdf > 0]
    simple_avg[prod_id] = max(float(dd.mean()), 0) if len(dd) > 0 else 0.0

## 6. Feature Engineering (131 Features)

In [ ]:
# Map Prophet predictions to grid
grid["prophet_yhat"] = 0.0
grid["prophet_trend"] = 0.0
grid["prophet_weekly"] = 0.0
grid["prophet_yearly"] = 0.0

lookup_records = []
for (day_val, prod_id), vals in prophet_predictions.items():
    lookup_records.append({
        "day": day_val, "id_produit": prod_id,
        "_pyhat": vals["yhat"], "_ptrend": vals["trend"],
        "_pweekly": vals["weekly"], "_pyearly": vals["yearly"],
    })

if lookup_records:
    lookup_df = pd.DataFrame(lookup_records)
    lookup_df["day"] = pd.to_datetime(lookup_df["day"])
    grid = grid.merge(lookup_df, on=["day", "id_produit"], how="left")
    for orig, tmp in [("prophet_yhat", "_pyhat"), ("prophet_trend", "_ptrend"),
                       ("prophet_weekly", "_pweekly"), ("prophet_yearly", "_pyearly")]:
        grid[orig] = grid[tmp].fillna(grid[orig])
        grid.drop(columns=tmp, inplace=True)

for prod_id, avg in simple_avg.items():
    mask = grid["id_produit"] == prod_id
    grid.loc[mask, "prophet_yhat"] = avg

print(f"Prophet yhat: mean={grid['prophet_yhat'].mean():.1f}, "
      f"median={grid['prophet_yhat'].median():.1f}")

In [ ]:
# Per-product demand statistics
prod_stats = daily.groupby("id_produit").agg(
    prod_avg_demand=("demand", "mean"),
    prod_med_demand=("demand", "median"),
    prod_std_demand=("demand", "std"),
    prod_n_days=("day", "nunique"),
).reset_index()
prod_stats["prod_std_demand"] = prod_stats["prod_std_demand"].fillna(1)
grid = grid.merge(prod_stats, on="id_produit", how="left")

# Lag features
g = grid.groupby("id_produit")["demand"]
for lag in [1, 2, 3, 7, 14, 21, 28]:
    grid[f"lag_{lag}"] = g.shift(lag)

# Rolling statistics
shifted = g.shift(1)
for w in [3, 7, 14, 28, 60]:
    roll = shifted.rolling(w, min_periods=1)
    grid[f"rmean_{w}"] = roll.mean()
    grid[f"rstd_{w}"]  = roll.std().fillna(0)
    if w in [7, 14, 28]:
        grid[f"rmax_{w}"] = roll.max()
        grid[f"rmin_{w}"] = roll.min()
        grid[f"rmed_{w}"] = roll.median()

for w in [7, 14, 28]:
    grid[f"rsum_{w}"] = shifted.rolling(w, min_periods=1).sum()

# Demand frequency rolling
gh = grid.groupby("id_produit")["has_demand"]
shifted_h = gh.shift(1)
for w in [7, 14, 28, 60]:
    grid[f"dfreq_{w}"] = shifted_h.rolling(w, min_periods=1).mean()

# Days since last demand
def _days_since(group):
    had = group["has_demand"].shift(1).values
    n = len(group)
    result = np.full(n, 999.0)
    last = -9999
    for i in range(n):
        if i > 0 and had[i - 1] == 1:
            last = i - 1
        if last >= 0:
            result[i] = float(i - last)
    return pd.Series(result, index=group.index)

grid["days_since"] = grid.groupby("id_produit", group_keys=False).apply(_days_since)

# EWM & CV
grid["cv_28"] = grid["rstd_28"] / (grid["rmean_28"] + 1e-6)
grid["ewm_7"]  = g.shift(1).transform(lambda x: x.ewm(span=7, min_periods=1).mean())
grid["ewm_28"] = g.shift(1).transform(lambda x: x.ewm(span=28, min_periods=1).mean())

# Calendar features + Fourier
grid["dow"] = grid["day"].dt.dayofweek.astype(float)
grid["month"] = grid["day"].dt.month.astype(float)
grid["week"] = grid["day"].dt.isocalendar().week.astype(float)
grid["is_wknd"] = (grid["dow"] >= 5).astype(float)
grid["dom"] = grid["day"].dt.day.astype(float)
grid["qtr"] = grid["day"].dt.quarter.astype(float)
grid["day_idx"] = (grid["day"] - grid["day"].min()).dt.days.astype(float)
grid["day_idx_sq"] = grid["day_idx"] ** 2 / 1e6

day_of_year = grid["day"].dt.dayofyear.astype(float)
for k in [1, 2, 3, 4]:
    grid[f"fourier_sin_y{k}"] = np.sin(2 * np.pi * k * day_of_year / 365.25)
    grid[f"fourier_cos_y{k}"] = np.cos(2 * np.pi * k * day_of_year / 365.25)
for k in [1, 2]:
    grid[f"fourier_sin_w{k}"] = np.sin(2 * np.pi * k * grid["dow"] / 7)
    grid[f"fourier_cos_w{k}"] = np.cos(2 * np.pi * k * grid["dow"] / 7)
for k in [1, 2]:
    grid[f"fourier_sin_m{k}"] = np.sin(2 * np.pi * k * grid["dom"] / 31)
    grid[f"fourier_cos_m{k}"] = np.cos(2 * np.pi * k * grid["dom"] / 31)

grid["is_month_start"] = (grid["dom"] <= 3).astype(float)
grid["is_month_end"] = (grid["dom"] >= 28).astype(float)
grid["is_week_start"] = (grid["dow"] == 0).astype(float)

print(f"Features created. Grid shape: {grid.shape}")

In [ ]:
# Prophet-derived features
pa = grid["prod_avg_demand"] + 1e-6
grid["prophet_ratio"] = grid["prophet_yhat"] / pa
grid["prophet_over_ewm7"] = grid["prophet_yhat"] / (grid["ewm_7"] + 1e-6)
grid["prophet_over_rmean7"] = grid["prophet_yhat"] / (grid["rmean_7"] + 1e-6)
grid["prophet_residual"] = grid["demand"] - grid["prophet_yhat"]
grid["prophet_resid_lag1"] = grid.groupby("id_produit")["prophet_residual"].shift(1)
grid["prophet_resid_rmean7"] = (grid.groupby("id_produit")["prophet_residual"]
                                  .shift(1).rolling(7, min_periods=1).mean())
grid["prophet_trend_norm"] = grid["prophet_trend"] / (pa + 1e-6)
grid["prophet_weekly_abs"] = grid["prophet_weekly"].abs()
grid["prophet_yearly_abs"] = grid["prophet_yearly"].abs()
grid["prophet_seasonal_str"] = grid["prophet_weekly_abs"] + grid["prophet_yearly_abs"]

# Normalized features
grid["lag1_norm"] = grid["lag_1"] / pa
grid["rmean7_norm"] = grid["rmean_7"] / pa
grid["rmean28_norm"] = grid["rmean_28"] / pa
grid["ewm7_norm"] = grid["ewm_7"] / pa
grid["ewm28_norm"] = grid["ewm_28"] / pa
grid["rmean7_over_28"] = grid["rmean_7"] / (grid["rmean_28"] + 1e-6)

# Product static features
pf = products[["id_produit", "total_demand", "demand_days",
               "avg_demand", "demand_frequency", "priority_score",
               "demand_score", "frequency_score"]].copy()
pf.columns = ["id_produit", "p_total", "p_days", "p_avg", "p_freq",
               "p_prio", "p_demand_score", "p_freq_score"]
grid = grid.merge(pf, on="id_produit", how="left")

# Category encoding
cat_map = products.set_index("id_produit")["categorie"]
grid["categorie"] = grid["id_produit"].map(cat_map).fillna("UNKNOWN")
grid["cat_enc"] = grid["categorie"].astype("category").cat.codes.astype(float)

# Segment x Temporal interactions
grid["hf_x_dow"] = grid["is_hf"] * grid["dow"]
grid["hf_x_month"] = grid["is_hf"] * grid["month"]
grid["hf_x_is_wknd"] = grid["is_hf"] * grid["is_wknd"]
grid["hf_x_lag1"] = grid["is_hf"] * grid["lag_1"].fillna(0)
grid["hf_x_rmean7"] = grid["is_hf"] * grid["rmean_7"].fillna(0)
grid["hf_x_prophet"] = grid["is_hf"] * grid["prophet_yhat"]

# Drop warm-up (60 days)
grid = grid[grid["day"] >= grid["day"].min() + pd.Timedelta(days=60)].copy()
grid.drop(columns=["prophet_residual"], inplace=True, errors="ignore")

# Clean infinities
for c in grid.select_dtypes(include=[np.number]).columns:
    grid[c] = grid[c].replace([np.inf, -np.inf], 0).fillna(0)

print(f"Final grid: {grid.shape}")

## 7. Train/Test Split & XGBoost Classifier

In [ ]:
# Feature list and temporal split
EXCLUDE_COLS = {"day", "id_produit", "demand", "has_demand", "categorie",
                "segment", "n_deliveries", "qty_delivered"}
FEATURE_COLS = sorted([c for c in grid.columns if c not in EXCLUDE_COLS])
print(f"Features: {len(FEATURE_COLS)}")

# Temporal split: last 30 days = test
split_date = grid["day"].max() - pd.Timedelta(days=30)
train_full = grid[grid["day"] <= split_date].copy()
test_full  = grid[grid["day"] >  split_date].copy()

train_full[FEATURE_COLS] = train_full[FEATURE_COLS].replace([np.inf, -np.inf], 0).fillna(0)
test_full[FEATURE_COLS]  = test_full[FEATURE_COLS].replace([np.inf, -np.inf], 0).fillna(0)

train_pos = train_full[train_full["has_demand"] == 1].copy()
test_pos  = test_full[test_full["has_demand"] == 1].copy()

print(f"Train: {len(train_full):,} total, {len(train_pos):,} demand rows")
print(f"Test:  {len(test_full):,} total, {len(test_pos):,} demand rows")
print(f"Split date: {split_date.date()}")

In [ ]:
# Train XGBoost binary classifier (demand yes/no)
y_tr_cls = train_full["has_demand"]
y_te_cls = test_full["has_demand"]
pos = y_tr_cls.sum()
neg = len(y_tr_cls) - pos

cls_params = {
    "objective": "binary:logistic",
    "eval_metric": "auc",
    "max_depth": 7,
    "learning_rate": 0.04,
    "subsample": 0.8,
    "colsample_bytree": 0.6,
    "colsample_bylevel": 0.8,
    "min_child_weight": 15,
    "scale_pos_weight": neg / max(pos, 1),
    "reg_alpha": 0.5,
    "reg_lambda": 3.0,
    "gamma": 0.1,
    "tree_method": "hist",
    "seed": 42,
}

d_tr_cls = xgb.DMatrix(train_full[FEATURE_COLS], label=y_tr_cls)
d_te_cls = xgb.DMatrix(test_full[FEATURE_COLS],  label=y_te_cls)

t0 = time.time()
classifier = xgb.train(
    cls_params, d_tr_cls, num_boost_round=800,
    evals=[(d_te_cls, "val")], early_stopping_rounds=50,
    verbose_eval=100,
)
print(f"\nTraining time: {time.time()-t0:.0f}s")
print(f"Best iteration: {classifier.best_iteration}")

proba_te = classifier.predict(d_te_cls)
auc = roc_auc_score(y_te_cls, proba_te)
print(f"Classifier AUC: {auc:.4f}")

## 8. Threshold, Blend & Bias Tuning

3D grid search over:
- **Threshold**: classifier probability cutoff for demand prediction
- **Alpha**: blend weight (Prophet vs EWM-7)
- **Bias multiplier**: global scaling factor

Optimization target: minimize WAPE + bias penalty

In [ ]:
# Evaluation setup
test_eval = test_full.reset_index(drop=True).copy()
d_te_all = xgb.DMatrix(test_eval[FEATURE_COLS])
proba_all = classifier.predict(d_te_all)
is_hf_test = test_eval["is_hf"].values
actual_all = test_eval["demand"].values
has_demand_mask = actual_all > 0
a_dd_sum = actual_all[has_demand_mask].sum()

prophet_base = test_eval["prophet_yhat"].fillna(0).values
ewm_base = test_eval["ewm_7"].fillna(0).values

def evaluate_fast(thr, alpha, bias_mult):
    pred_mask = proba_all >= thr
    pred_demand = np.zeros(len(test_eval))
    if pred_mask.sum() > 0:
        blended = alpha * prophet_base[pred_mask] + (1 - alpha) * ewm_base[pred_mask]
        pred_demand[pred_mask] = blended * bias_mult
    p_dd = pred_demand[has_demand_mask]
    a_dd = actual_all[has_demand_mask]
    if a_dd_sum == 0:
        return 999, 0
    wape = np.sum(np.abs(a_dd - p_dd)) / a_dd_sum * 100
    bias = (p_dd.sum() - a_dd_sum) / a_dd_sum * 100
    return wape, bias

# Coarse grid search
print("Coarse grid search...")
best_score = 1e9
best_thr, best_alpha, best_bm = 0.1, 0.7, 1.0

for thr in np.arange(0.02, 0.50, 0.02):
    for alpha in np.arange(0.0, 1.05, 0.1):
        for bm in np.arange(0.5, 2.0, 0.1):
            w, b = evaluate_fast(thr, alpha, bm)
            bias_pen = abs(b) * 3 if b < 0 else (max(0, b - 5) * 3)
            score = w + bias_pen
            if score < best_score:
                best_score = score
                best_thr, best_alpha, best_bm = thr, alpha, bm

print(f"Coarse best: thr={best_thr:.2f}, alpha={best_alpha:.2f}, bm={best_bm:.2f}")

# Fine grid search
print("Fine grid search...")
for thr in np.arange(max(0.02, best_thr - 0.04), min(0.50, best_thr + 0.04), 0.005):
    for alpha in np.arange(max(0.0, best_alpha - 0.15), min(1.05, best_alpha + 0.15), 0.025):
        for bm in np.arange(max(0.5, best_bm - 0.2), min(2.0, best_bm + 0.2), 0.01):
            w, b = evaluate_fast(thr, alpha, bm)
            bias_pen = abs(b) * 3 if b < 0 else (max(0, b - 5) * 3)
            score = w + bias_pen
            if score < best_score:
                best_score = score
                best_thr, best_alpha, best_bm = thr, alpha, bm

w_final, b_final = evaluate_fast(best_thr, best_alpha, best_bm)
print(f"\n" + "="*60)
print(f"BEST PARAMETERS:")
print(f"  Threshold:    {best_thr:.3f}")
print(f"  Alpha:        {best_alpha:.3f}")
print(f"  Bias mult:    {best_bm:.3f}")
print(f"  WAPE (dd):    {w_final:.2f}%")
print(f"  Bias (dd):    {b_final:.2f}%")
print("="*60)

## 9. Final Evaluation

In [ ]:
# Full evaluation with best parameters
pred_mask = proba_all >= best_thr
pred_final = np.zeros(len(test_eval))
if pred_mask.sum() > 0:
    blended = best_alpha * prophet_base[pred_mask] + (1 - best_alpha) * ewm_base[pred_mask]
    pred_final[pred_mask] = blended * best_bm

actual_full = test_eval["demand"].values
has_demand = actual_full > 0

# Demand-days metrics
a_dd = actual_full[has_demand]
p_dd = pred_final[has_demand]
wape_dd = np.sum(np.abs(a_dd - p_dd)) / a_dd.sum() * 100
bias_dd = (p_dd.sum() - a_dd.sum()) / a_dd.sum() * 100

# Full-grid metrics
wape_grid = np.sum(np.abs(actual_full - pred_final)) / actual_full.sum() * 100
bias_grid = (pred_final.sum() - actual_full.sum()) / actual_full.sum() * 100

# Baselines
bl_lag1 = test_eval["lag_1"].fillna(0).values
bl_lag1_dd = np.sum(np.abs(a_dd - bl_lag1[has_demand])) / a_dd.sum() * 100
bl_ewm = test_eval["ewm_7"].fillna(0).values
bl_ewm_dd = np.sum(np.abs(a_dd - bl_ewm[has_demand])) / a_dd.sum() * 100
bl_prophet_dd = np.sum(np.abs(a_dd - prophet_base[has_demand])) / a_dd.sum() * 100

print("="*60)
print("FINAL EVALUATION RESULTS")
print("="*60)
print(f"  Demand-Days  WAPE : {wape_dd:.2f}%  (target < 50%)")
print(f"  Demand-Days  Bias : {bias_dd:.2f}%  (target 0-5%)")
print(f"  Full-Grid    WAPE : {wape_grid:.1f}%")
print(f"  Full-Grid    Bias : {bias_grid:.1f}%")
print(f"")
print(f"  Baseline Lag-1 (dd):   {bl_lag1_dd:.1f}%")
print(f"  Baseline EWM-7 (dd):   {bl_ewm_dd:.1f}%")
print(f"  Baseline Prophet (dd): {bl_prophet_dd:.1f}%")
print(f"")
improv = (bl_lag1_dd - wape_dd) / bl_lag1_dd * 100
print(f"  Improvement vs Lag-1:  {improv:.1f}%")
print("="*60)

# Per-segment evaluation
for seg_name, seg_val in [("HF", 1.0), ("LF", 0.0)]:
    seg_mask = has_demand & (is_hf_test == seg_val)
    if seg_mask.sum() > 0:
        a_s = actual_full[seg_mask]
        p_s = pred_final[seg_mask]
        sw = np.sum(np.abs(a_s - p_s)) / a_s.sum() * 100
        sb = (p_s.sum() - a_s.sum()) / a_s.sum() * 100
        print(f"  {seg_name} demand-days: WAPE={sw:.1f}%, Bias={sb:.1f}%, n={seg_mask.sum()}")

## 10. Save Models

In [ ]:
# Save XGBoost classifier
classifier.save_model(str(MODEL / "xgboost_classifier_model.json"))

# Train and save minimal regressor (for API compatibility)
y_tr_reg = np.log1p(train_pos["demand"].values)
y_te_reg = np.log1p(test_pos["demand"].values)
reg_params = {
    "objective": "reg:squarederror", "eval_metric": "mae",
    "max_depth": 4, "learning_rate": 0.05, "subsample": 0.8,
    "colsample_bytree": 0.6, "min_child_weight": 20,
    "reg_alpha": 1.0, "reg_lambda": 5.0, "tree_method": "hist", "seed": 42,
}
d_tr_reg = xgb.DMatrix(train_pos[FEATURE_COLS], label=y_tr_reg)
d_te_reg = xgb.DMatrix(test_pos[FEATURE_COLS],  label=y_te_reg)
regressor = xgb.train(
    reg_params, d_tr_reg, num_boost_round=500,
    evals=[(d_te_reg, "val")], early_stopping_rounds=50, verbose_eval=0,
)
regressor.save_model(str(MODEL / "xgboost_regression_model.json"))

# Save Prophet metadata
with open(MODEL / "prophet_meta.json", "w") as f:
    json.dump({
        "prophet_models": {str(k): v for k, v in prophet_models_meta.items()},
        "simple_avg": {str(k): v for k, v in simple_avg.items()},
        "segment_map": {str(k): v for k, v in seg_map.items()},
    }, f, indent=2)

# Save product attributes
prod_attrs = products_raw.set_index("id_produit").to_dict("index")
with open(MODEL / "product_attributes.json", "w") as f:
    json.dump({str(k): v for k, v in prod_attrs.items()}, f, indent=2)

# Save delivery stats
del_stats_dict = prod_del_stats.set_index("id_produit").to_dict("index")
with open(MODEL / "delivery_stats.json", "w") as f:
    json.dump({str(k): v for k, v in del_stats_dict.items()}, f, indent=2)

# Save category encoding
cat_enc_map = dict(zip(
    grid["categorie"].astype("category").cat.categories,
    range(len(grid["categorie"].astype("category").cat.categories))
))
with open(MODEL / "cat_encoding.json", "w") as f:
    json.dump(cat_enc_map, f, indent=2)

# Save config
config = {
    "optimal_threshold": round(float(best_thr), 4),
    "bias_multiplier": round(float(best_bm), 4),
    "ensemble_alpha": round(float(best_alpha), 4),
    "feature_cols_regression": FEATURE_COLS,
    "model_type": "prophet_classifier_blend_v9",
    "performance": {
        "wape_demand_days": round(float(wape_dd), 2),
        "bias_demand_days": round(float(bias_dd), 2),
        "wape_full_grid": round(float(wape_grid), 2),
        "bias_full_grid": round(float(bias_grid), 2),
        "baseline_lag1_wape_dd": round(float(bl_lag1_dd), 2),
        "baseline_ewm7_wape_dd": round(float(bl_ewm_dd), 2),
        "baseline_prophet_wape_dd": round(float(bl_prophet_dd), 2),
        "improvement_vs_lag1_pct": round(float(improv), 1),
        "classifier_auc": round(float(auc), 4),
    },
}
with open(MODEL / "forecast_config.json", "w") as f:
    json.dump(config, f, indent=2)

print("All models saved to:", MODEL.resolve())
print("Files saved:")
for f in MODEL.glob("*.json"):
    print(f"  {f.name}: {f.stat().st_size / 1024:.0f} KB")

## 11. Task 1: Warehouse Optimization Approach

Task 1 uses a **heuristic/algorithmic approach** (no ML model needed):

### Storage Assignment Strategy
1. **Segment-aware placement**: HF products -> PICKING zone (close to expedition), LF products -> RESERVE zone
2. **Scoring function**: `score = -dist_to_expedition * w1 - floor * w2` (HF gets higher distance weight)
3. **Heavy item handling**: Extra ground floor penalty for products > 5kg
4. **State tracking**: In-memory slot occupancy prevents double-assignment

### Picking Optimization Strategy
1. **Multi-chariot splitting**: Items split across chariots by weight capacity (300kg max)
2. **Nearest-neighbor routing**: Each chariot follows nearest-unvisited-location path
3. **Congestion detection**: Zone visit counting warns when >3 picks in same zone
4. **Distance calculation**: Manhattan 3D distance (x, y, z) between locations

### Simulation Engine
- Processes events chronologically
- Ingoing: assigns optimal storage with demand forecasting context
- Outgoing: picks from stored location, generates route, releases slot
- Tracks warehouse state throughout

Implementation: See `inference_optimization.py` and `main.py`

In [ ]:
# Demonstrate warehouse optimization with sample data
warehouse_locations = pd.read_csv(DATA / "warehouse_locations.csv")

print("=== Warehouse Layout ===")
print(f"Total locations: {len(warehouse_locations)}")
print(f"\nBy type:")
print(warehouse_locations['type_emplacement'].value_counts())
print(f"\nBy zone:")
print(warehouse_locations['zone'].value_counts())
print(f"\nFloor distribution:")
print(warehouse_locations['z'].value_counts().sort_index())

In [ ]:
# Demonstrate storage assignment logic
print("=== Storage Assignment Demo ===")
print("")

# HF Product example
hf_product = 31554
hf_info = products[products['id_produit'] == hf_product].iloc[0]
seg = segments[segments['id_produit'] == hf_product]['segment'].values[0]
print(f"Product {hf_product} (Segment: {seg}):")
print(f"  Demand frequency: {hf_info['demand_frequency']:.4f}")
print(f"  Avg demand: {hf_info['avg_demand']:.0f}")
print(f"  -> PLACEMENT: PICKING zone (close to expedition)")

print("")

# LF Product example
lf_product = 31339
lf_info = products[products['id_produit'] == lf_product].iloc[0]
seg = segments[segments['id_produit'] == lf_product]['segment'].values[0]
print(f"Product {lf_product} (Segment: {seg}):")
print(f"  Demand frequency: {lf_info['demand_frequency']:.4f}")
print(f"  Avg demand: {lf_info['avg_demand']:.0f}")
print(f"  -> PLACEMENT: RESERVE zone")

## Summary

### Model Files Produced
| File | Size | Description |
|------|------|-------------|
| `xgboost_classifier_model.json` | ~2 MB | Binary demand classifier |
| `xgboost_regression_model.json` | ~1 MB | Regressor (API compat.) |
| `prophet_meta.json` | ~200 KB | Per-SKU Prophet params (571 products) |
| `forecast_config.json` | ~10 KB | Threshold, blend, features |
| `product_attributes.json` | ~100 KB | Physical product attributes |
| `delivery_stats.json` | ~50 KB | Delivery pattern stats |
| `cat_encoding.json` | ~1 KB | Category encoding map |

### All Models < 5 MB total (lightweight)

### Inference Scripts
- `inference_forecast.py`: Task 2 standalone inference
- `inference_optimization.py`: Task 1 standalone inference
- `main.py`: Full API with all endpoints